# 04 - Results Analysis and Visualization

This notebook provides comprehensive analysis and visualization of evaluation results.

## Objectives
- Load and compare multiple evaluation runs
- Perform deep analysis of performance patterns
- Generate publication-quality visualizations
- Export insights and recommendations

In [ ]:
# Import required libraries
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import logging
from datetime import datetime
import json
from collections import defaultdict

# Add src to path
sys.path.append('../src')

# Import raglab modules
from core.io import RunManager
from eval import load_evaluation_results

# Set up logging and plotting
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

plt.style.use('default')
sns.set_palette("husl")

print("✅ Imports successful")

## Discover and Load Evaluation Runs

In [ ]:
# Discover available evaluation runs
run_manager = RunManager(runs_base_path='../runs')
available_runs = run_manager.list_runs()

print(f"🔍 Found {len(available_runs)} evaluation runs:")
for i, run_name in enumerate(available_runs):
    print(f"   {i+1}. {run_name}")

if not available_runs:
    print("⚠️  No evaluation runs found. Please run notebooks 02 or 03 first.")
else:
    print(f"\n📊 Analysis will focus on these runs")

In [ ]:
# Load all evaluation runs
run_data = {}
run_base_path = Path('../runs')

for run_name in available_runs:
    run_path = run_base_path / run_name
    
    try:
        # Load config, outputs, and metrics
        config = run_manager.load_config(str(run_path))
        outputs, metrics = load_evaluation_results(str(run_path))
        
        run_data[run_name] = {
            'config': config,
            'outputs': outputs,
            'metrics': metrics
        }
        
        print(f"✅ Loaded {run_name}: {len(outputs)} examples")
        
    except Exception as e:
        print(f"❌ Failed to load {run_name}: {e}")

print(f"\n📚 Successfully loaded {len(run_data)} evaluation runs")

## Run Comparison Analysis

In [ ]:
# Create comparison summary
if len(run_data) > 1:
    print("📊 Run Comparison Summary:")
    print("-" * 100)
    
    comparison_data = []
    
    for run_name, data in run_data.items():
        metrics = data['metrics']
        config = data['config']
        
        row = {
            'run_name': run_name,
            'type': config.get('evaluation_type', 'unknown'),
            'examples': metrics.get('total_examples', 0),
            'error_rate': metrics.get('error_rate', 0)
        }
        
        # Add BEIR metrics if available
        if 'beir_metrics' in metrics:
            beir = metrics['beir_metrics']
            row.update({
                'avg_recall': beir.get('recall_at_k', {}).get('mean', 0),
                'avg_precision': beir.get('precision_at_k', {}).get('mean', 0),
                'avg_ndcg': beir.get('ndcg_at_k', {}).get('mean', 0)
            })
        
        # Add judge metrics if available
        if 'judge_metrics' in metrics:
            judge = metrics['judge_metrics']
            row.update({
                'correctness_rate': judge.get('correctness_rate', 0),
                'hallucination_rate': judge.get('hallucination_rate', 0)
            })
        
        # Add meta-eval metrics if available
        if 'meta_eval_metrics' in metrics:
            meta = metrics['meta_eval_metrics']
            row['judge_accuracy'] = meta.get('judge_accuracy', 0)
        
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
else:
    print("📊 Single run analysis (comparison requires multiple runs)")
    comparison_df = None

## Performance Trend Analysis

In [ ]:
# Analyze performance trends across runs
if comparison_df is not None and len(comparison_df) > 1:
    
    # Sort runs by timestamp (extract from run name)
    comparison_df['timestamp'] = comparison_df['run_name'].str.extract(r'(\d{4}-\d{2}-\d{2}_\d{4})')
    comparison_df = comparison_df.sort_values('timestamp')
    
    # Create performance trend visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. BEIR Metrics Trend
    if 'avg_recall' in comparison_df.columns:
        axes[0,0].plot(range(len(comparison_df)), comparison_df['avg_recall'], 'o-', label='Recall@K', linewidth=2)
        axes[0,0].plot(range(len(comparison_df)), comparison_df['avg_precision'], 's-', label='Precision@K', linewidth=2)
        axes[0,0].plot(range(len(comparison_df)), comparison_df['avg_ndcg'], '^-', label='nDCG@K', linewidth=2)
        axes[0,0].set_title('BEIR Metrics Trend')
        axes[0,0].set_xlabel('Run Number')
        axes[0,0].set_ylabel('Score')
        axes[0,0].legend()
        axes[0,0].grid(True, alpha=0.3)
        axes[0,0].set_xticks(range(len(comparison_df)))
        axes[0,0].set_xticklabels([f"Run {i+1}" for i in range(len(comparison_df))])
    else:
        axes[0,0].text(0.5, 0.5, 'No BEIR Metrics\nAvailable', ha='center', va='center', transform=axes[0,0].transAxes)
    
    # 2. Judge Performance Trend
    if 'correctness_rate' in comparison_df.columns:
        axes[0,1].plot(range(len(comparison_df)), comparison_df['correctness_rate'], 'o-', label='Correctness Rate', linewidth=2, color='green')
        axes[0,1].plot(range(len(comparison_df)), 1-comparison_df['hallucination_rate'], 's-', label='No Hallucination Rate', linewidth=2, color='blue')
        if 'judge_accuracy' in comparison_df.columns:
            axes[0,1].plot(range(len(comparison_df)), comparison_df['judge_accuracy'], '^-', label='Judge Accuracy', linewidth=2, color='purple')
        axes[0,1].set_title('Judge Performance Trend')
        axes[0,1].set_xlabel('Run Number')
        axes[0,1].set_ylabel('Rate')
        axes[0,1].legend()
        axes[0,1].grid(True, alpha=0.3)
        axes[0,1].set_xticks(range(len(comparison_df)))
        axes[0,1].set_xticklabels([f"Run {i+1}" for i in range(len(comparison_df))])
    else:
        axes[0,1].text(0.5, 0.5, 'No Judge Metrics\nAvailable', ha='center', va='center', transform=axes[0,1].transAxes)
    
    # 3. Error Rate Trend
    axes[1,0].plot(range(len(comparison_df)), comparison_df['error_rate'], 'o-', linewidth=2, color='red')
    axes[1,0].set_title('Error Rate Trend')
    axes[1,0].set_xlabel('Run Number')
    axes[1,0].set_ylabel('Error Rate')
    axes[1,0].grid(True, alpha=0.3)
    axes[1,0].set_xticks(range(len(comparison_df)))
    axes[1,0].set_xticklabels([f"Run {i+1}" for i in range(len(comparison_df))])
    
    # 4. Run Summary Bar Chart
    metric_cols = [col for col in ['avg_recall', 'correctness_rate', 'judge_accuracy'] if col in comparison_df.columns]
    if metric_cols:
        x = np.arange(len(comparison_df))
        width = 0.2
        
        for i, col in enumerate(metric_cols):
            axes[1,1].bar(x + i*width, comparison_df[col], width, label=col.replace('_', ' ').title())
        
        axes[1,1].set_title('Performance Summary')
        axes[1,1].set_xlabel('Run Number')
        axes[1,1].set_ylabel('Score')
        axes[1,1].set_xticks(x + width)
        axes[1,1].set_xticklabels([f"Run {i+1}" for i in range(len(comparison_df))])
        axes[1,1].legend()
        axes[1,1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.suptitle('Evaluation Performance Trends', fontsize=16, fontweight='bold', y=1.02)
    plt.show()
    
else:
    print("⚠️  Trend analysis requires multiple runs")

## Deep Dive Analysis

In [ ]:
# Select the most recent or best performing run for deep analysis
if run_data:
    # Choose the most recent run (or you can modify this logic)
    selected_run = max(run_data.keys())
    selected_data = run_data[selected_run]
    
    print(f"🔍 Deep Analysis of: {selected_run}")
    print(f"   Type: {selected_data['config'].get('evaluation_type', 'unknown')}")
    print(f"   Examples: {len(selected_data['outputs'])}")
    
    # Convert outputs to DataFrame for detailed analysis
    detailed_df = pd.DataFrame(selected_data['outputs'])
    
    # Extract nested data if this is a complete agent evaluation
    if 'judge_output' in detailed_df.columns:
        # Extract judge metrics
        judge_metrics = pd.json_normalize(detailed_df['judge_output'])
        meta_metrics = pd.json_normalize(detailed_df['meta_eval_output'])
        beir_metrics = pd.json_normalize(detailed_df['beir_metrics'])
        
        # Combine into analysis DataFrame
        analysis_df = pd.concat([
            detailed_df[['example_id']],
            judge_metrics.add_prefix('judge_'),
            meta_metrics.add_prefix('meta_'),
            beir_metrics.add_prefix('beir_')
        ], axis=1)
        
        print(f"\n📊 Analysis DataFrame created with {len(analysis_df)} examples and {len(analysis_df.columns)} metrics")
        
    else:
        analysis_df = detailed_df
        print(f"\n📊 Using raw outputs for analysis")
else:
    print("❌ No runs available for deep analysis")
    analysis_df = None

In [ ]:
# Performance pattern analysis
if analysis_df is not None and 'judge_correctness_binary' in analysis_df.columns:
    
    print("🎯 Performance Pattern Analysis:")
    
    # Correlation analysis
    numeric_cols = analysis_df.select_dtypes(include=[np.number]).columns
    correlation_matrix = analysis_df[numeric_cols].corr()
    
    # Create correlation heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": .5}, fmt='.2f')
    plt.title('Performance Metrics Correlation Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Identify strong correlations
    strong_correlations = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            corr = correlation_matrix.iloc[i, j]
            if abs(corr) > 0.5:  # Strong correlation threshold
                col1 = correlation_matrix.columns[i]
                col2 = correlation_matrix.columns[j]
                strong_correlations.append((col1, col2, corr))
    
    if strong_correlations:
        print(f"\n🔗 Strong Correlations (|r| > 0.5):")
        for col1, col2, corr in sorted(strong_correlations, key=lambda x: abs(x[2]), reverse=True):
            print(f"   {col1} ↔ {col2}: {corr:.3f}")
    else:
        print(f"\n📊 No strong correlations found")

## Failure Analysis

In [ ]:
# Analyze failure patterns
if analysis_df is not None and 'judge_correctness_binary' in analysis_df.columns:
    
    print("❌ Failure Analysis:")
    
    # Identify failed examples
    failed_examples = analysis_df[~analysis_df['judge_correctness_binary']]
    successful_examples = analysis_df[analysis_df['judge_correctness_binary']]
    
    print(f"   Failed examples: {len(failed_examples)}")
    print(f"   Successful examples: {len(successful_examples)}")
    
    if len(failed_examples) > 0:
        # Compare failed vs successful examples
        print(f"\n📊 Failed vs Successful Comparison:")
        
        comparison_metrics = ['beir_recall_at_k', 'beir_precision_at_k', 'beir_ndcg_at_k']
        available_metrics = [m for m in comparison_metrics if m in analysis_df.columns]
        
        if available_metrics:
            fig, axes = plt.subplots(1, len(available_metrics), figsize=(5*len(available_metrics), 5))
            if len(available_metrics) == 1:
                axes = [axes]
            
            for i, metric in enumerate(available_metrics):
                # Box plot comparison
                data_to_plot = [
                    failed_examples[metric].dropna(),
                    successful_examples[metric].dropna()
                ]
                labels = ['Failed', 'Successful']
                
                box_plot = axes[i].boxplot(data_to_plot, labels=labels, patch_artist=True)
                axes[i].set_title(f'{metric.replace("_", " ").title()}')
                axes[i].set_ylabel('Score')
                axes[i].grid(True, alpha=0.3)
                
                # Color boxes
                colors = ['lightcoral', 'lightgreen']
                for patch, color in zip(box_plot['boxes'], colors):
                    patch.set_facecolor(color)
                
                # Add statistical comparison
                failed_mean = failed_examples[metric].mean()
                success_mean = successful_examples[metric].mean()
                axes[i].text(0.5, 0.95, f'Failed: {failed_mean:.3f}\nSuccess: {success_mean:.3f}', 
                           transform=axes[i].transAxes, ha='center', va='top', 
                           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
            
            plt.tight_layout()
            plt.suptitle('Failed vs Successful Examples Comparison', fontsize=14, fontweight='bold', y=1.02)
            plt.show()
            
        # Show specific failed examples
        print(f"\n🔍 Failed Examples Details:")
        for _, row in failed_examples.iterrows():
            print(f"   {row['example_id']}:")
            if 'beir_recall_at_k' in row:
                print(f"     Retrieval: R={row['beir_recall_at_k']:.3f}, P={row['beir_precision_at_k']:.3f}")
            if 'judge_hallucination_binary' in row:
                print(f"     Hallucination: {row['judge_hallucination_binary']}")
            if 'meta_judge_correct' in row:
                print(f"     Judge Reliable: {row['meta_judge_correct']}")
    
    else:
        print("✅ No failed examples found!")

## Risk Analysis (Insurance Specific)

In [ ]:
# Analyze insurance risk patterns
if analysis_df is not None and 'judge_risk_direction' in analysis_df.columns:
    
    risk_data = analysis_df['judge_risk_direction'].dropna()
    impact_data = analysis_df['judge_risk_impact'].dropna()
    
    if len(risk_data) > 0:
        print("🏥 Insurance Risk Analysis:")
        
        # Risk direction distribution
        risk_counts = risk_data.value_counts().sort_index()
        risk_labels = {
            -1: 'Care Avoidance Risk\n(Overestimated Cost)',
            0: 'No Clear Risk Direction',
            1: 'Unexpected Cost Risk\n(Underestimated Cost)'
        }
        
        # Create risk analysis visualization
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Risk direction pie chart
        labels = [risk_labels.get(idx, f'Risk {idx}') for idx in risk_counts.index]
        colors = ['lightcoral', 'lightgray', 'orange']
        risk_colors = [colors[idx+1] for idx in risk_counts.index]  # Map -1,0,1 to 0,1,2
        
        axes[0].pie(risk_counts.values, labels=labels, autopct='%1.1f%%', colors=risk_colors)
        axes[0].set_title('Risk Direction Distribution', fontweight='bold')
        
        # Risk impact distribution
        if len(impact_data) > 0:
            impact_counts = impact_data.value_counts().sort_index()
            impact_labels = ['Minimal (0)', 'Low (1)', 'Moderate (2)', 'High (3)']
            
            bars = axes[1].bar(impact_counts.index, impact_counts.values, 
                             color=['lightgreen', 'yellow', 'orange', 'red'])
            axes[1].set_title('Risk Impact Distribution', fontweight='bold')
            axes[1].set_xlabel('Impact Level')
            axes[1].set_ylabel('Count')
            axes[1].set_xticks(impact_counts.index)
            axes[1].set_xticklabels([impact_labels[i] for i in impact_counts.index])
            
            # Add count labels on bars
            for bar, count in zip(bars, impact_counts.values):
                axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                           str(count), ha='center', va='bottom')
        else:
            axes[1].text(0.5, 0.5, 'No Risk Impact\nData Available', 
                        ha='center', va='center', transform=axes[1].transAxes)
        
        plt.tight_layout()
        plt.show()
        
        # Risk statistics
        print(f"\n📊 Risk Statistics:")
        total_risk_cases = len(risk_data)
        care_avoidance = sum(risk_data == -1)
        unexpected_cost = sum(risk_data == 1)
        no_clear_direction = sum(risk_data == 0)
        
        print(f"   Total risk cases: {total_risk_cases}")
        print(f"   Care avoidance risk: {care_avoidance} ({care_avoidance/total_risk_cases:.1%})")
        print(f"   Unexpected cost risk: {unexpected_cost} ({unexpected_cost/total_risk_cases:.1%})")
        print(f"   No clear direction: {no_clear_direction} ({no_clear_direction/total_risk_cases:.1%})")
        
        if len(impact_data) > 0:
            avg_impact = impact_data.mean()
            high_impact = sum(impact_data >= 2)
            print(f"   Average impact level: {avg_impact:.2f}")
            print(f"   High impact cases (≥2): {high_impact} ({high_impact/len(impact_data):.1%})")
            
    else:
        print("📊 No insurance risk data available in this evaluation")

## Export Results and Insights

In [ ]:
# Generate comprehensive insights report
insights_report = {
    'analysis_timestamp': datetime.now().isoformat(),
    'runs_analyzed': len(run_data),
    'insights': [],
    'recommendations': []
}

if run_data:
    # Overall insights
    if comparison_df is not None and len(comparison_df) > 1:
        # Multi-run insights
        best_run = comparison_df.loc[comparison_df['avg_recall'].idxmax()] if 'avg_recall' in comparison_df.columns else None
        if best_run is not None:
            insights_report['insights'].append(f"Best performing run: {best_run['run_name']} with {best_run['avg_recall']:.3f} recall@k")
        
        # Performance trends
        if 'avg_recall' in comparison_df.columns:
            recall_trend = comparison_df['avg_recall'].diff().mean()
            if recall_trend > 0.01:
                insights_report['insights'].append("Positive retrieval performance trend observed across runs")
            elif recall_trend < -0.01:
                insights_report['insights'].append("Declining retrieval performance trend - investigate causes")
    
    # Single run insights
    if analysis_df is not None:
        # Performance insights
        if 'judge_correctness_binary' in analysis_df.columns:
            correctness_rate = analysis_df['judge_correctness_binary'].mean()
            insights_report['insights'].append(f"Overall correctness rate: {correctness_rate:.1%}")
            
            if correctness_rate < 0.7:
                insights_report['recommendations'].append("Improve generation quality - consider better prompts or models")
        
        if 'beir_recall_at_k' in analysis_df.columns:
            avg_recall = analysis_df['beir_recall_at_k'].mean()
            insights_report['insights'].append(f"Average retrieval recall: {avg_recall:.3f}")
            
            if avg_recall < 0.7:
                insights_report['recommendations'].append("Improve retrieval system - consider different embeddings or chunking strategies")
        
        # Risk insights
        if 'judge_risk_direction' in analysis_df.columns:
            risk_data = analysis_df['judge_risk_direction'].dropna()
            if len(risk_data) > 0:
                care_avoidance_rate = sum(risk_data == -1) / len(risk_data)
                unexpected_cost_rate = sum(risk_data == 1) / len(risk_data)
                
                insights_report['insights'].append(f"Care avoidance risk rate: {care_avoidance_rate:.1%}")
                insights_report['insights'].append(f"Unexpected cost risk rate: {unexpected_cost_rate:.1%}")
                
                if care_avoidance_rate > 0.3:
                    insights_report['recommendations'].append("High care avoidance risk - review cost estimation accuracy")
                if unexpected_cost_rate > 0.3:
                    insights_report['recommendations'].append("High unexpected cost risk - ensure conservative cost estimates")

# Save insights report
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
insights_file = f'../artifacts/insights_report_{timestamp}.json'

with open(insights_file, 'w') as f:
    json.dump(insights_report, f, indent=2)

print(f"📊 Analysis Complete!")
print(f"\n💡 Key Insights:")
for insight in insights_report['insights']:
    print(f"   • {insight}")

print(f"\n🎯 Recommendations:")
for rec in insights_report['recommendations']:
    print(f"   • {rec}")

print(f"\n📁 Full insights report saved to: {insights_file}")
print(f"🎉 Analysis notebook complete!")